In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image
import io
import cv2

In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml

--2020-08-23 10:18:39--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.05s   

2020-08-23 10:18:39 (18.0 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]



In [ ]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
FACE_DESC, FACE_NAME = pickle.load(open('trainset.pk', 'rb'))

VideoCapture()
eval_js('create()')
while True:
    byte = eval_js('capture()')
    frame = byte2image(byte)
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        img = frame[y-10:y+h+10, x-10:x+w+10][:,:,::-1]
        dets = detector(img, 1)
        for k, d in enumerate(dets):
            shape = sp(img, d)
            face_desc0 = model.compute_face_descriptor(img, shape, 1)
            d = []
            for face_desc in FACE_DESC:
                d.append(np.linalg.norm(np.array(face_desc) - np.array(face_desc0)))
            d = np.array(d)
            idx = np.argmin(d)
            if d[idx] < 0.5:
                name = FACE_NAME[idx]
                #print(name)
                cv2.putText(frame, name, (x, y-5), cv2.FONT_HERSHEY_COMPLEX, .7, (255,255,255),2)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    eval_js('showimg("{}")'.format(image2byte(frame)))

In [ ]:
import tensorflow as tf
from keras.preprocessing import image


EMOTIONS_LIST = ["Angry", "Disgust",
                     "Fear", "Happy",
                     "Sad", "Surprise",
                     "Neutral"]

Age_LIST = ["Adult", "Kid",
                     "Senior", "Teenager"]

Gender_LIST = ["Male", "Female"]


def LoadModel(model_json_file, model_weights_file):
   # load model from JSON file
    with open(model_json_file, "r") as json_file:
         loaded_model_json = json_file.read()
         loaded_model = tf.keras.models.model_from_json(loaded_model_json)
    return loaded_model_json,loaded_model


emotion_model_json,emotional_loaded_model=LoadModel("/content/drive/My Drive/Bootcamp/face_model.json","/content/drive/My Drive/Bootcamp/face_model.h5")

age_model_json,age_loaded_model=LoadModel("/content/drive/My Drive/Bootcamp/modelAge.json","/content/drive/My Drive/Bootcamp/modelAge.h5")

gender_model_json,gender_loaded_model=LoadModel("/content/drive/My Drive/Bootcamp/modelGender.json","/content/drive/My Drive/Bootcamp/modelGender.h5")

def predict_emotion(img,loaded_model):
    preds = loaded_model.predict(img,batch_size=10)
    #print(EMOTIONS_LIST[np.argmax(preds)])
    return EMOTIONS_LIST[np.argmax(preds)]


def predict_age(img,loaded_model):
    preds = loaded_model.predict(img,batch_size=10)
    #print(Age_LIST[np.argmax(preds)])
    return Age_LIST[np.argmax(preds)]


def predict_gender(img,loaded_model):
    preds = loaded_model.predict(img,batch_size=10)
    print(preds)
    print(Gender_LIST[np.argmax(preds)])
    return Gender_LIST[np.argmax(preds)]

In [ ]:
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#detector = dlib.get_frontal_face_detector()
#sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
#model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
#FACE_DESC, FACE_NAME = pickle.load(open('trainset.pk', 'rb'))

VideoCapture()
eval_js('create()')
while True:
    byte = eval_js('capture()')
    frame = byte2image(byte)
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    faces = face_detector.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        #img = frame[y-10:y+h+10, x-10:x+w+10][:,:,::-1]
        fc = gray[y:y+h, x:x+w]

        roi = cv2.resize(frame, (150, 150))
        k = image.img_to_array(roi)
        k = np.expand_dims(k, axis=0)
        images = np.vstack([k])
        #pred = model.predict_emotion(images)

        #predict Gender
        gender=predict_gender(images,gender_loaded_model)

        #predict Age
        age=predict_age(images,age_loaded_model)

        roi = cv2.resize(fc, (48, 48))
        #pred = cnn.predict_emotion(roi[np.newaxis, :, :, np.newaxis])

        # predict Emotion
        emotion=predict_emotion(roi[np.newaxis, :, :, np.newaxis],emotional_loaded_model)

        Result= gender + " " + age + " " + emotion


        cv2.putText(frame, Result, (x, y-5), cv2.FONT_HERSHEY_COMPLEX, .7, (255,255,255),2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    eval_js('showimg("{}")'.format(image2byte(frame)))

MessageError: ignored